In [1]:
%reset -f
!pyclean .

Cleaning directory .
Total 2 files, 1 directories removed.


In [2]:
from lalala import *
%matplotlib inline

In [3]:
no_bg_meta = pd.read_csv(os.path.join(DATA_DIRECTORY,"no_background","no_background.csv"),index_col="image_id")
no_bg_meta.head(1)

,spacecraft_id,xmin,ymin,xmax,ymax
image_id,,,,,
23c7f89d00006caee536a22e5807de95,5,994,596,1079,705


In [4]:
spacecraft_5 = [os.path.join(DATA_DIRECTORY,"no_background", "images", i) + ".png" for i in no_bg_meta[no_bg_meta.spacecraft_id == 5].index]
spacecraft_5_bbox = [[k["xmin"], k["ymin"], k["xmax"], k["ymax"]] for i,k in no_bg_meta.iterrows()if k["spacecraft_id"] == 5]

In [5]:
space5 = dataset_flow(spacecraft_5, spacecraft_5_bbox, batch_size=2)
space5
image_sample, bbox_sample = next(space5.__iter__())

# Backbone, no need to train

## Initialize the Backbone and using the size


In [6]:
# Initialize the Backbone and using the size
backbone = resnet_fpn((224,224,3))

## create the FPN elements

In [7]:
fpn_feature_maps = backbone.predict(image_sample)

1/1 [==============================] - 4s 4s/step


In [23]:
image_sample.shape

TensorShape([2, 224, 224, 3])

## input the FPN into RPN and get the probability and bbox

In [8]:
fpn_feature_maps[0].shape

(2, 56, 56, 16)

In [9]:
def generate_anchors(fpn_feature_maps, ratios = [0.5, 1., 2.], scales = [2 ** (x/3) for x in range(3)], feature_size =[56., 28., 14., 7., 4.]):
    anchors_all = []
    for ix, fpn_feature_map in enumerate(fpn_feature_maps):
        base_size = fpn_feature_map.shape[1]
        anchors = []
        for i in range(fpn_feature_map.shape[-1]):
            anchors1 = []
            for scale in scales:
                for ratio in ratios:
                    # Calculate anchor dimensions based on scale and ratio
                    anchor_height = base_size * scale * np.sqrt(ratio) /2
                    anchor_width = base_size * scale / np.sqrt(ratio) /2
                    
                    # Generate anchors across the feature map
                    
                    x = feature_size[ix]
                    y = feature_size[ix]
                    center_x = (x + 0.5) /2
                    center_y = (y + 0.5) /2
                    anchors1.append([
                        int(max(center_x - anchor_width / 2, 0.)),
                        int(max(center_y - anchor_height / 2, 0.)),
                        int(min(center_x + anchor_width / 2, x)),
                        int(min(center_y + anchor_height / 2, x)),
                    ])
            anchors.append(anchors1)
    
        anchors_all.append(anchors)

    return np.array(anchors_all)

In [10]:
anchors = generate_anchors(fpn_feature_maps)

In [11]:
fpn_feature_maps[0][0,:,:,1].shape

(56, 56)

In [12]:
anchors.shape

(5, 16, 9, 4)

In [13]:
anchor = anchors[0, 1, 1, :]
fpn_feature_maps[1][0,:,:,1].shape

(28, 28)

In [14]:
anchor

array([14, 14, 42, 42])

In [15]:
def crop_regions_from_feature_map(feature_map, anchors):
    crops = []
    for i in range(anchors.shape[0]): #5 # Iterate through all the anchors
        for j in range(anchors.shape[1]): # 256   # Iterate through each of the 9 bboxes for the anchor
            for k in range(anchors.shape[2]):   # 9
                anchor = anchors[i, j, k, :]
                crop = tf.image.crop_to_bounding_box(tf.expand_dims(feature_map[i][0,:,:,j], -1),
                                                    offset_height=anchor[0],
                                                    offset_width=anchor[1],
                                                    target_height=anchor[2] - anchor[0],
                                                    target_width=anchor[3] - anchor[1])
                crop = np.squeeze(crop, axis=-1)
                crops.append(crop)
    return crops

# Adjust anchors if necessary to fit your specific scenario
cropped_regions = crop_regions_from_feature_map(fpn_feature_maps, anchors)
#keep in mind that the array has different shape

## RoI Align

In [16]:
len(cropped_regions)

720

In [17]:
def roi_align(cropped_regions):
    # Normalize proposals from [0, 1] to feature map scale
    pooled_list = []
    for cropped_region in cropped_regions:
        cropped_region = tf.expand_dims(cropped_region, axis = -1)
        resized_data = tf.image.resize(cropped_region, [224, 224], method=tf.image.ResizeMethod.BILINEAR)
        pooled_list.append(resized_data)
        
    return tf.concat(pooled_list, axis=-1)

In [18]:
roiali = roi_align(cropped_regions)

In [19]:
roiali.shape

TensorShape([224, 224, 720])

## create a big data generator

In [20]:
spacecraft_5 = [os.path.join(DATA_DIRECTORY,"no_background", "images", i) + ".png" for i in no_bg_meta[no_bg_meta.spacecraft_id == 5].index]
spacecraft_5_bbox = [[k["xmin"], k["ymin"], k["xmax"], k["ymax"]] for i,k in no_bg_meta.iterrows()if k["spacecraft_id"] == 5]

In [ ]:
def dataset_flow(image_path : list, bbox : list, batch_size : int):
    bbox = [[i[0]/1280*224,i[1]/1024*224,i[2]/1280*224,i[3]/1024*224] for i in bbox]
    dataset = tf.data.Dataset.from_tensor_slices((image_path, bbox))
    #<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))>
    dataset = dataset.map(load_image_and_labels).batch(batch_size)
    return dataset

## Three models for Computer Vision
1. Mask Using Unet
2. BBOX using CNN + FC
3. Classification using CNN + FC